In [2]:
import subprocess
import tempfile
import os
import pickle

def run_vsflow(filename_and_content):
    input_file_name, file_content = filename_and_content

    print("got here")
    # Write the file content to a temporary input file
    with tempfile.NamedTemporaryFile(delete=False, mode="w+", suffix=".sdf") as tmp_input_file:
        tmp_input_file.write(file_content)
        tmp_input_file.flush()
        input_file_path = tmp_input_file.name

    # Create a temporary output file
    with tempfile.NamedTemporaryFile(delete=False, mode="wb+", suffix=".vsdb") as tmp_output_file:
        temp_file_path = os.path.abspath(tmp_output_file.name)

    # The vsflow command uses the temporary input file
    command = ["vsflow", "preparedb", "-i", input_file_path, "-o", temp_file_path, "-s", "-can", "-np", "4"]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    pickle_data = None

    if result.returncode == 0:
        print("vsflow executed successfully.")
        if os.path.getsize(temp_file_path) > 0:
            with open(temp_file_path, 'rb') as f:
                pickle_data = pickle.dumps(f.read())
        else:
            print(f"The output file {temp_file_path} is empty.")
    else:
        print("vsflow failed to execute:")
        print(result.stdout)
        print(result.stderr)
    try:
        os.remove(input_file_path)
    except OSError as e:
        print(f"Error: {input_file_path} : {e.strerror}")

    return input_file_name, pickle_data

In [3]:
destination_directory = "outputs"

input_files = ["fda.sdf",
               "fda copy.sdf", 
               "fda copy 2.sdf", 
               "fda copy 3.sdf", 
               "fda copy 4.sdf", 
               "fda copy 5.sdf",
               "fda copy 6.sdf",
               "fda copy 7.sdf",
               "fda copy 8.sdf",
               "fda copy 9.sdf",
               "fda copy 10.sdf",
               "fda copy 11.sdf",
               "fda copy 12.sdf",
               "fda copy 13.sdf",
               "fda copy 14.sdf",
               "fda copy 15.sdf",
               "fda copy 16.sdf",
               "fda copy 17.sdf",
               "fda copy 18.sdf",
               "fda copy 19.sdf",
               ]



file_contents_with_names = []
for input_file in input_files:
    with open(input_file, 'r') as f:
        file_content = f.read()
        file_contents_with_names.append((input_file, file_content))

In [3]:
for pickle_object in  map(run_vsflow, file_contents_with_names):
    input_file_name, object = pickle_object
    with open(os.path.join(destination_directory, os.path.splitext(input_file_name)[0] + ".vsdb"), 'wb') as out_file:
        out_file.write(object)
    break

got here
vsflow executed successfully.


In [ ]:
# import coiled

# coiled.create_software_environment(
#     name="vsflow_env",
#     conda={
#         "channels": ["conda-forge"],
#         "dependencies": ["python=3.9", 
#                          "dask[complete]", 
#                          "coiled",
#                          "rdkit",
#                          "fpdf",
#                          "pdfrw",
#                          "xlsxwriter",
#                          "xlrd",
#                          "pymol-open-source",
#                          "molvs",
#                          "matplotlib",
#         ]
#     },
#     pip=[
#         "git+https://github.com/czodrowskilab/VSFlow.git",
#     ],
# )


In [4]:

cluster = coiled.Cluster(n_workers=10,
                         name = "testing",
                         software = "vsflow")
client = cluster.get_client()


# from dask.distributed import LocalCluster

# cluster = LocalCluster(n_workers=3)

# client = cluster.get_client()

Output()

In [5]:
client

<Client: 'tls://10.0.43.49:8786' processes=10 threads=40, memory=148.43 GiB>

In [6]:
file_contents_with_names = client.scatter(file_contents_with_names)

pickle_object_futures = client.map(run_vsflow, file_contents_with_names)

from dask.distributed import as_completed

for future in as_completed(pickle_object_futures):
    input_file_name, object = future.result()
    with open(os.path.join(destination_directory, os.path.splitext(input_file_name)[0] + ".vsdb"), 'wb') as out_file:
        out_file.write(object)

TypeError: a bytes-like object is required, not 'NoneType'

In [7]:
# client.restart()